# TRANSFORMACIÓN DE DATOS

## IMPORTAR PAQUETES

In [237]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.preprocessing import OneHotEncoder
from category_encoders import TargetEncoder

#Automcompletar rápido
%config IPCompleter.greedy=True

## IMPORTAR LOS DATOS

1.- Sustituir la ruta del proyecto.

In [238]:
ruta_proyecto = '../../'

2.- Nombrar los ficheros de datos.

In [239]:
nombre_cat = 'cat_resultado_eda.pickle'
nombre_num = 'num_resultado_eda.pickle'

3.- Cargar los datos.

In [240]:
cat = pd.read_pickle(ruta_proyecto + '/02_Datos/03_Trabajo/' + nombre_cat)
num = pd.read_pickle(ruta_proyecto + '/02_Datos/03_Trabajo/' + nombre_num)

## NUEVAS VARIABLES

En este caso ya vienen creadas:

* componentes de la fecha
* variables de calendario

Voy a crear:

* las que indentifique en EDA
* lags
* ventanas móviles

Para esta fase necesito volver a unir en un solo dataframe.

In [241]:
df = pd.concat([cat,num], axis = 1)
df

,warehouse_id,item_id,d,year,month,wday,weekday,event_name_1,event_type_1,wm_yr_wk,ventas,sell_price
date,,,,,,,,,,,,
2013-01-01,CA_3,BLACK_3_090,d_704,2013,1,4,Tuesday,NewYear,National,11249,0,1.25
2013-01-01,CA_3,BLACK_3_120,d_704,2013,1,4,Tuesday,NewYear,National,11249,0,4.98
2013-01-01,CA_3,BLACK_3_202,d_704,2013,1,4,Tuesday,NewYear,National,11249,20,4.28
2013-01-01,CA_3,BLACK_3_252,d_704,2013,1,4,Tuesday,NewYear,National,11249,34,1.48
2013-01-01,CA_3,BLACK_3_288,d_704,2013,1,4,Tuesday,NewYear,National,11249,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
2015-11-30,CA_4,BLACK_3_329,d_1767,2015,11,3,Monday,Sin_evento,Sin_evento,11544,5,2.98
2015-11-30,CA_4,BLACK_3_555,d_1767,2015,11,3,Monday,Sin_evento,Sin_evento,11544,4,1.68
2015-11-30,CA_4,BLACK_3_586,d_1767,2015,11,3,Monday,Sin_evento,Sin_evento,11544,9,1.68


### Variable demanda intermitente

Esta variable va a identificar cuantos días seguidos han transcurrido con ventas cero.

La defino como que si los últimos n días han tenido cero ventas entonces hay rotura de stock.

Puedo crear varias cambiando el n.

Servirá para modelizar.

#### nota

Podria calcular la probabilidad con estadistica una variable predeterminada que exista 0 ventas o haya roturas de stock o usar ML para predecir esto....


Aun asi, lo importante tambien es ser resolutivo, poner el tiempo justo y necesario, no quiero alargar el proyecto

**Practicidad**

Creo la funcion rotura_stock

Basicamente lo que hacer primero 

* cero_ventas: crea el indicado 0 ventas, si las ventas son 0 pon un indicador 1

Luego se hace una ventana movil para contar en N dias cuantas ventas a habido

* rotura_stock: si en esos ultimso 5 dias ha habido 0 ventas entonces me pone un indicador 1

Se crea 3 variables de rotura de tock con dias 3 7 15 para que se pueda ver los diferentes patrones, de mas rapido a mas lento

Esto es uan variable que tiene en cuenta los datos del mismo dia, es decir el modelo no va a tener esta informacion, porque trabaja sobre el propio dia, por eso creo despues unos lags para que el modelo lo pueda hacer sobre el dia anterior

In [242]:
def rotura_stock(ventas, n = 5):
    cero_ventas = pd.Series(np.where(ventas == 0,1,0))
    num_ceros = cero_ventas.rolling(n).sum()
    rotura_stock = np.where(num_ceros == n,1,0)
    return(rotura_stock)

In [243]:
df = df.sort_values(by = ['warehouse_id','item_id','date'])

In [244]:
df['rotura_stock_3'] = df.groupby(['warehouse_id','item_id']).ventas.transform(lambda x: rotura_stock(x, 3)).values

In [245]:
df['rotura_stock_7'] = df.groupby(['warehouse_id','item_id']).ventas.transform(lambda x: rotura_stock(x,7)).values

In [246]:
df['rotura_stock_15'] = df.groupby(['warehouse_id','item_id']).ventas.transform(lambda x: rotura_stock(x,15)).values

### Variables de lag

Creo lags sobre las siguientes variables:

* ventas: lags de 15 días
* sell_price: lags de 7 días
* rotura_stock: lag de un día

#### Nota teorica

En el precio algo la variable mas corta por si baja puntualmente el precio. 

Para la rotura de stock un lag de un dia, esto es importante, esto es por la recurrencia, la rotura stock sale de variable ventas, los modelos de ml unicamente puede predecir un dia, entonces encadeno predicciones, hago esa prediccion de ventas para mañana. pero se coge un dia atras para hacer la prediccion. 

In [247]:
def crear_lags(df, variable, num_lags = 7):
    
    #Crea el objeto dataframe
    lags = pd.DataFrame()
    
    #Crea todos los lags
    for cada in range(1,num_lags+1):
        lags[variable + '_lag_'+ str(cada)] = df[variable].shift(cada)
    
    #Devuelve el dataframe de lags
    return(lags)

In [248]:
lags_sell_price_df = df.groupby(['warehouse_id','item_id'])\
                    .apply(lambda x: crear_lags(df = x, variable = 'sell_price', num_lags= 7))

In [249]:
lags_rotura_stock_3_df = df.groupby(['warehouse_id','item_id'])\
                    .apply(lambda x: crear_lags(df = x, variable = 'rotura_stock_3', num_lags= 1))

In [250]:
lags_rotura_stock_7_df = df.groupby(['warehouse_id','item_id'])\
                    .apply(lambda x: crear_lags(df = x, variable = 'rotura_stock_7', num_lags= 1))

In [251]:
lags_rotura_stock_15_df = df.groupby(['warehouse_id','item_id'])\
                    .apply(lambda x: crear_lags(df = x, variable = 'rotura_stock_15', num_lags= 1))

In [252]:
lags_ventas_df = df.groupby(['warehouse_id','item_id'])\
                    .apply(lambda x: crear_lags(df = x, variable = 'ventas', num_lags= 15))

### Variables de ventanas móviles

Creo tres tipos de ventanas móviles sobre las ventas:

* mínimo móvil
* media móvil
* máximo móvil

Cada uno de ellos en el rango de 15 días.

#### nota

Hago minimos moviles media moviles y maximos moviles para ver si alguna de esas variables puede ser predictiva y ayudar a los modelos

In [253]:
def min_movil(df, variable, num_periodos = 7):

    minm = pd.DataFrame()
    
    for cada in range(2,num_periodos+1): #Empieza por dos para poder calcular la media de algo, no hay media de 1
        minm[variable + '_minm_' + str(cada)] = df[variable].shift(1).rolling(cada).min()
    
    return(minm)

In [254]:
def media_movil(df, variable, num_periodos = 7):

    mm = pd.DataFrame()
    
    for cada in range(2,num_periodos+1):
        mm[variable + '_mm_' + str(cada)] = df[variable].shift(1).rolling(cada).mean()

    return(mm)

In [255]:
def max_movil(df, variable, num_periodos = 7):

    maxm = pd.DataFrame()
    
    for cada in range(2,num_periodos+1):
        maxm[variable + '_maxm_' + str(cada)] = df[variable].shift(1).rolling(cada).max()

    return(maxm)

In [256]:
min_movil_df = df.groupby(['warehouse_id','item_id'])\
                    .apply(lambda x: min_movil(df = x, variable = 'ventas', num_periodos= 15))

In [257]:
media_movil_df = df.groupby(['warehouse_id','item_id'])\
                    .apply(lambda x: media_movil(df = x, variable = 'ventas', num_periodos= 15))

In [258]:
max_movil_df = df.groupby(['warehouse_id','item_id'])\
                    .apply(lambda x: max_movil(df = x, variable = 'ventas', num_periodos= 15))

## PREPARAR LOS DATASETS

### Unir todos los dataframes generados

In [259]:
df = df.reset_index(drop=False)
lags_sell_price_df = lags_sell_price_df.reset_index(drop=True)
lags_rotura_stock_3_df = lags_rotura_stock_3_df.reset_index(drop=True)
lags_rotura_stock_7_df = lags_rotura_stock_3_df.reset_index(drop=True)
lags_rotura_stock_15_df = lags_rotura_stock_3_df.reset_index(drop=True)
lags_ventas_df = lags_ventas_df.reset_index(drop=True)
min_movil_df = min_movil_df.reset_index(drop=True)
media_movil_df = media_movil_df.reset_index(drop=True)
max_movil_df = max_movil_df.reset_index(drop=True)

In [260]:
df_unido = pd.concat([df,
                      lags_sell_price_df,
                      lags_rotura_stock_3_df,
                      lags_rotura_stock_7_df,
                      lags_rotura_stock_15_df,
                      lags_ventas_df,
                      min_movil_df,
                      media_movil_df,
                      max_movil_df], axis = 1)

df_unido = df_unido.set_index('date')

In [261]:
df_unido

,warehouse_id,item_id,d,year,month,wday,weekday,event_name_1,event_type_1,wm_yr_wk,...,ventas_maxm_6,ventas_maxm_7,ventas_maxm_8,ventas_maxm_9,ventas_maxm_10,ventas_maxm_11,ventas_maxm_12,ventas_maxm_13,ventas_maxm_14,ventas_maxm_15
date,,,,,,,,,,,,,,,,,,,,,
2013-01-01,CA_3,BLACK_3_090,d_704,2013,1,4,Tuesday,NewYear,National,11249,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-01-02,CA_3,BLACK_3_090,d_705,2013,1,5,Wednesday,Sin_evento,Sin_evento,11249,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-01-03,CA_3,BLACK_3_090,d_706,2013,1,6,Thursday,Sin_evento,Sin_evento,11249,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-01-04,CA_3,BLACK_3_090,d_707,2013,1,7,Friday,Sin_evento,Sin_evento,11249,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-01-05,CA_3,BLACK_3_090,d_708,2013,1,1,Saturday,Sin_evento,Sin_evento,11250,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2015-11-26,CA_4,BLACK_3_714,d_1763,2015,11,6,Thursday,Thanksgiving,National,11543,...,15.0,15.0,15.0,15.0,15.0,17.0,17.0,17.0,17.0,17.0
2015-11-27,CA_4,BLACK_3_714,d_1764,2015,11,7,Friday,Sin_evento,Sin_evento,11543,...,22.0,22.0,22.0,22.0,22.0,22.0,22.0,22.0,22.0,22.0
2015-11-28,CA_4,BLACK_3_714,d_1765,2015,11,1,Saturday,Sin_evento,Sin_evento,11544,...,22.0,22.0,22.0,22.0,22.0,22.0,22.0,22.0,22.0,22.0


### Eliminar los nulos que han generado las nuevas variables

Los primeros registros tienen nulos en las nuevas variables con las que hemos creado "lags", los elimino

In [262]:
df_unido.dropna(inplace=True)

### Eliminar las variables que no voy a necesitar para modelizar

In [263]:
a_eliminar = ['d','wm_yr_wk','sell_price','rotura_stock_3','rotura_stock_7','rotura_stock_15']

* sell_price: no uso esta, usaré los lags sobre el precio de venta
* rotura_stock: usaré el lag de cada que he creado

In [264]:
df_unido.drop(columns=a_eliminar, inplace=True)

### Identificar la target

Voy a hacer target encoding

In [265]:
target = df_unido.ventas

### Separar num y cat

In [266]:
cat = df_unido.select_dtypes(include='O')

In [267]:
num = df_unido.select_dtypes(exclude='O')

## TRANSFORMACIÓN DE CATEGÓRICAS

### One Hot Encoding

Aplicar a todas las variables categoricas nominales(no orden)

#### Variables a aplicar OHE

In [268]:
var_ohe = ['year',
          'month',
          'wday',
          'weekday',
          'event_name_1',
          'event_type_1'
        ]

#### Instanciar

In [269]:
ohe = OneHotEncoder(sparse = False, handle_unknown='ignore')

#### Entrenar y aplicar

In [270]:
cat_ohe = ohe.fit_transform(cat[var_ohe])

C:\Users\sallus\AppData\Local\miniconda3\envs\forecastalfarben\Lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


#### Guardar como dataframe

In [271]:
cat_ohe = pd.DataFrame(cat_ohe, columns = ohe.get_feature_names_out())

### Target Encoding

Si un producto se vende mas un dia que otro, esa media que se vende el producto cada dia, puede ser muy predictivo. 

Es como si vendiera costillas un finde, es mas predictivo tenerlo.

#### Variables a aplicar TE

In [272]:
var_te = ['year',
          'month',
          'wday',
          'weekday',
          'event_name_1',
          'event_type_1'
        ]

#### Instanciar

In [273]:
te = TargetEncoder(min_samples_leaf=100, return_df = False)

#### Entrenar y aplicar

In [274]:
cat_te = te.fit_transform(cat[var_te], y = target)

#### Guardar como dataframe

In [275]:
#Añadir sufijos a los nombres
nombres_te = [variable + '_te' for variable in var_te]

#Guardar como dataframe
cat_te = pd.DataFrame(cat_te, columns = nombres_te)

## UNIFICAR DATASETS TRANSFORMADOS

### Meter en una lista todos los dataframes generados

Rescato de df_unido las variables de segmentación.

In [276]:
de_df_unido = df_unido[['warehouse_id','item_id']].reset_index()

de_df_unido.head(2)

,date,warehouse_id,item_id
0,2013-01-16,CA_3,BLACK_3_090
1,2013-01-17,CA_3,BLACK_3_090


### Unir todos los dataframes

In [279]:
dataframes = [de_df_unido, cat_ohe,cat_te,num.reset_index(drop=True)]

In [280]:
df_tablon = pd.concat(dataframes, axis = 1)

df_tablon

,date,warehouse_id,item_id,year_2013,year_2014,year_2015,month_1,month_2,month_3,month_4,...,ventas_maxm_6,ventas_maxm_7,ventas_maxm_8,ventas_maxm_9,ventas_maxm_10,ventas_maxm_11,ventas_maxm_12,ventas_maxm_13,ventas_maxm_14,ventas_maxm_15
0,2013-01-16,CA_3,BLACK_3_090,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,351.0,351.0,351.0,351.0,351.0,351.0,351.0,351.0,351.0,351.0
1,2013-01-17,CA_3,BLACK_3_090,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,351.0,351.0,351.0,351.0,351.0,351.0,351.0,351.0,351.0,351.0
2,2013-01-18,CA_3,BLACK_3_090,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,351.0,351.0,351.0,351.0,351.0,351.0,351.0,351.0,351.0,351.0
3,2013-01-19,CA_3,BLACK_3_090,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,281.0,351.0,351.0,351.0,351.0,351.0,351.0,351.0,351.0,351.0
4,2013-01-20,CA_3,BLACK_3_090,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,362.0,362.0,362.0,362.0,362.0,362.0,362.0,362.0,362.0,362.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20920,2015-11-26,CA_4,BLACK_3_714,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,15.0,15.0,15.0,15.0,15.0,17.0,17.0,17.0,17.0,17.0
20921,2015-11-27,CA_4,BLACK_3_714,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,22.0,22.0,22.0,22.0,22.0,22.0,22.0,22.0,22.0,22.0
20922,2015-11-28,CA_4,BLACK_3_714,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,22.0,22.0,22.0,22.0,22.0,22.0,22.0,22.0,22.0,22.0
20923,2015-11-29,CA_4,BLACK_3_714,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,22.0,22.0,22.0,22.0,22.0,22.0,22.0,22.0,22.0,22.0


## GUARDAR DATASET TRAS TRANSFORMACIÓN DE DATOS

En formato pickle para no perder las modificaciones de metadatos.

In [281]:
ruta_df_tablon = ruta_proyecto + '/02_Datos/03_Trabajo/' + 'df_tablon.pickle'

In [282]:
df_tablon.to_pickle(ruta_df_tablon)